# Clustering
**Teori** - [**Notion Clustering**](https://www.notion.so/mercantec/Machine-Learning-e89a2baf0d414172b13d07465366482e?pvs=4#2c11e4d51b994deda5dcdbcbb5063c2c)

# Clustering med manuel lighedsmåling

I denne del vil du gruppere chokolader i datasættet [Chocolate Bar Ratings](https://www.kaggle.com/rtatman/chocolate-bar-ratings) ved hjælp af k-means clustering-algoritmen med en manuel lighedsmåling. Datasættet indeholder bedømmelser af chokoladebarer sammen med deres kakaoindhold, bønnetype, bøndeoprindelse, producentnavn og producentland. Du vil:

* Indlæse og rengøre data.
* Behandle data.
* Beregne ligheden mellem par af chokolader.
* Cluster chokolader ved hjælp af k-means.
* Kontrollere Clusteranalysens resultat ved hjælp af kvalitetsmålinger.


# 1. Indlæs og rens data

Kør følgende celle for at indlæse og rense chokoladedatasættet. Du behøver ikke at
forstå koden. De første få rækker af datasættet vises. Undersøg
funktionerne og deres værdier.


In [ ]:
import math

from matplotlib import pyplot as plt
import numpy as np
import numpy.linalg as nla
import pandas as pd
import seaborn as sns
import altair as alt
import re
import pdb  # for Python debugger
import sys
from os.path import join

# Set the output display to have one digit for decimal places and limit it to
# printing 15 rows.
np.set_printoptions(precision=2)
pd.options.display.float_format = '{:.2f}'.format
pd.options.display.max_rows = 15

choc_data = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/flavors_of_cacao.csv", sep=",", encoding='latin-1')

# We can rename the columns.
choc_data.columns = ['maker', 'specific_origin', 'reference_number', 'review_date', 'cocoa_percent', 'maker_location', 'rating', 'bean_type', 'broad_origin']

# choc_data.dtypes

# Replace empty/null values with "Blend"
choc_data['bean_type'] = choc_data['bean_type'].fillna('Blend')

#@title Cast bean_type to string to remove leading 'u'
choc_data['bean_type'] = choc_data['bean_type'].astype(str)
choc_data['cocoa_percent'] = choc_data['cocoa_percent'].str.strip('%')
choc_data['cocoa_percent'] = pd.to_numeric(choc_data['cocoa_percent'])

#@title Correct spelling mistakes, and replace city with country name
choc_data['maker_location'] = choc_data['maker_location']\
.str.replace('Amsterdam', 'Holland')\
.str.replace('U.K.', 'England')\
.str.replace('Niacragua', 'Nicaragua')\
.str.replace('Domincan Republic', 'Dominican Republic')

# Adding this so that Holland and Netherlands map to the same country.
choc_data['maker_location'] = choc_data['maker_location']\
.str.replace('Holland', 'Netherlands')

def cleanup_spelling_abbrev(text):
    replacements = [
        ['-', ', '], ['/ ', ', '], ['/', ', '], ['\(', ', '], [' and', ', '], [' &', ', '], ['\)', ''],
        ['Dom Rep|DR|Domin Rep|Dominican Rep,|Domincan Republic', 'Dominican Republic'],
        ['Mad,|Mad$', 'Madagascar, '],
        ['PNG', 'Papua New Guinea, '],
        ['Guat,|Guat$', 'Guatemala, '],
        ['Ven,|Ven$|Venez,|Venez$', 'Venezuela, '],
        ['Ecu,|Ecu$|Ecuad,|Ecuad$', 'Ecuador, '],
        ['Nic,|Nic$', 'Nicaragua, '],
        ['Cost Rica', 'Costa Rica'],
        ['Mex,|Mex$', 'Mexico, '],
        ['Jam,|Jam$', 'Jamaica, '],
        ['Haw,|Haw$', 'Hawaii, '],
        ['Gre,|Gre$', 'Grenada, '],
        ['Tri,|Tri$', 'Trinidad, '],
        ['C Am', 'Central America'],
        ['S America', 'South America'],
        [', $', ''], [',  ', ', '], [', ,', ', '], ['\xa0', ' '],[',\s+', ','],
        [' Bali', ',Bali']
    ]
    for i, j in replacements:
        text = re.sub(i, j, text)
    return text

choc_data['specific_origin'] = choc_data['specific_origin'].str.replace('.', '').apply(cleanup_spelling_abbrev)

#@title Cast specific_origin to string
choc_data['specific_origin'] = choc_data['specific_origin'].astype(str)

#@title Replace null-valued fields with the same value as for specific_origin
choc_data['broad_origin'] = choc_data['broad_origin'].fillna(choc_data['specific_origin'])

#@title Clean up spelling mistakes and deal with abbreviations
choc_data['broad_origin'] = choc_data['broad_origin'].str.replace('.', '').apply(cleanup_spelling_abbrev)

# Change 'Trinitario, Criollo' to "Criollo, Trinitario"
# Check with choc_data['bean_type'].unique()
choc_data.loc[choc_data['bean_type'].isin(['Trinitario, Criollo']),'bean_type'] = "Criollo, Trinitario"
# Confirm with choc_data[choc_data['bean_type'].isin(['Trinitario, Criollo'])]

# Fix chocolate maker names
choc_data.loc[choc_data['maker']=='Shattel','maker'] = 'Shattell'
choc_data['maker'] = choc_data['maker'].str.replace(u'Na\xef\xbf\xbdve','Naive')

# Save the original column names
original_cols = choc_data.columns.values

choc_data.head()

# 2. Forbehandling af data
Du vil forbehandle dine data ved hjælp af de teknikker, der beskrives i
[Forbered Data](https://developers.google.com/machine-learning/clustering/prepare-data).

Lad os starte med trækken `review_date`. Hvis du antager, at fremstillingen af chokolade
ikke ændrede sig i løbet af de 10 års data, har `review_date` ingen korrelation
med chokoladen selv. Du kan trygt ignorere denne egenskab. Dog, som en god datavidenskabsmand, bør du være nysgerrig om dine data. Lad os
plotte fordelingen for `review date` ved hjælp af en funktion fra Seaborn-datavisualiseringsbiblioteket. Det ser ud til, at ingen spiste chokolade i 2009 og 2013. Dog er den samlede tendens for chokoladespisning positiv og meget opmuntrende. Dette er en god
tid til at nyde lidt chokolade selv!


In [ ]:
sns.distplot(choc_data['review_date'])

Plottet for fordelingen af `rating`. Overvej, hvordan du ville behandle denne fordeling. Fortsæt derefter for svaret.

In [ ]:
# check the distribution
sns.distplot(choc_data['rating'])

Fordelingen af `rating` minder groft om en Gaussisk fordeling. Hvordan behandles Gaussiske fordelinger? Du ved det. Normaliser dataen.


In [ ]:
# its a Gaussian! So, use z-score to normalize the data
choc_data['rating_norm'] = (choc_data['rating'] - choc_data['rating'].mean()
                           ) / choc_data['rating'].std()

Undersøg fordelingen af `cocoa_percent` og overvej, hvordan den skal behandles. Tjek derefter nedenfor for svaret.


In [ ]:
sns.distplot(choc_data['cocoa_percent'])

Fordelingen af `cocoa_percent` er tæt nok på en Gaussisk fordeling. Normaliser dataen.


In [ ]:
choc_data['cocoa_percent_norm'] = (
    choc_data['cocoa_percent'] -
    choc_data['cocoa_percent'].mean()) / choc_data['cocoa_percent'].std()

Vis de første rækker for at kontrollere normaliseringen af `rating` og `cocoa_percent`.


In [ ]:
choc_data.head()

Du har kakaobønnernes oprindelsesland i `broad_origin` og chokoladens produktionsland i `maker_location`. Men for at beregne ligheden har du brug for landenes længde- og breddegrader. Heldigvis er denne geografiske information tilgængelig i en anden tabel på
developers.google.com! Følgende kode downloader Dataset Publishing Language (DSPL)
Countries-tabel og sammenkæder den med vores tabel med chokoladeanmeldelser ved hjælp af landets
navn som nøgle. Bemærk, at du tilnærmer lande ved længde- og breddegraderne for deres centre.

Vis de første rækker for at kontrollere
det forarbejdede data. Bemærk de nyoprettede felter `maker_lat`, `maker_long`, `origin_lat` og `origin_long`. Stemmer værdierne i felterne overens med dine forventninger?


In [ ]:
#@title Run code to add latitude and longitude data
# Load lat long data

countries_info = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/countries_lat_long.csv", sep=",", encoding='latin-1')

#Join the chocolate review and geographic information tables on maker country name
choc_data = pd.merge(
    choc_data, countries_info, left_on="maker_location", right_on="name")
choc_data.rename(
    columns={
        "longitude": "maker_long",
        "latitude": "maker_lat"
    }, inplace=True)
choc_data.drop(
    columns=["name", "country"], inplace=True)  # don't need this data

#Join the chocolate review and geographic information tables on origin country name
choc_data = pd.merge(
    choc_data, countries_info, left_on="broad_origin", right_on="name")
choc_data.rename(
    columns={
        "longitude": "origin_long",
        "latitude": "origin_lat"
    },
    inplace=True)
choc_data.drop(
    columns=["name", "country"], inplace=True)  # don't need this data

choc_data.head()

Kontroller fordelingen af bredde- og længdegrader og overvej, hvordan fordelingerne skal behandles. Tjek derefter nedenfor for svaret.

In [ ]:
sns.distplot(choc_data['maker_lat'])

Da bredde- og længdegrader ikke følger en bestemt fordeling, konverter latitude- og longitude-informationen til kvantiler. Vis de sidste rækker for at bekræfte kvantilværdierne.


In [ ]:
numQuantiles = 20
colsQuantiles = ['maker_lat', 'maker_long', 'origin_lat', 'origin_long']

def createQuantiles(dfColumn, numQuantiles):
  return pd.qcut(dfColumn, numQuantiles, labels=False, duplicates='drop')


for string in colsQuantiles:
  choc_data[string] = createQuantiles(choc_data[string], numQuantiles)

choc_data.tail()

Kvantilværdierne spænder op til 20. Før kvantilværdierne til den samme skala som andre trækekendetegn ved at skalere dem til [0,1].


In [ ]:
def minMaxScaler(numArr):
  minx = np.min(numArr)
  maxx = np.max(numArr)
  numArr = (numArr - minx) / (maxx - minx)
  return numArr


for string in colsQuantiles:
  choc_data[string] = minMaxScaler(choc_data[string])

Trækkene `maker` og `bean_type` er kategoriske træk. Konverter
kategoriske træk til one-hot encoding.

In [ ]:
# duplicate the "maker" feature since it's removed by one-hot encoding function
choc_data['maker2'] = choc_data['maker']
choc_data = pd.get_dummies(choc_data, columns=['maker2'], prefix=['maker'])
# similarly, duplicate the "bean_type" feature
choc_data['bean_type2'] = choc_data['bean_type']
choc_data = pd.get_dummies(choc_data, columns=['bean_type2'], prefix=['bean'])

Efter clustering, når du fortolker resultaterne, kan det forarbejdede trækdata være
svært at læse. Gem de oprindelige trækdata i en ny dataramme, så du kan
referere til dem senere. Behold kun de forarbejdede data i `choc_data`.


# Split dataframe into two frames: Original data and data for clustering
choc_data_backup = choc_data.loc[:, original_cols].copy(deep=True)
choc_data.drop(columns=original_cols, inplace=True)

# get_dummies returned ints for one-hot encoding but we want floats so divide by
# 1.0
# Note: In the latest version of "get_dummies", you can set "dtype" to float
choc_data = choc_data / 1.0

Gennemgå de sidste poster for at sikre, at dine dyrebare chokoladedata ser
godt ud! Husk, at `choc_data` kun viser kolonner med forarbejdede data, da de kolonner, der indeholder de originale data, blev flyttet til `choc_data_backup`.


In [ ]:
choc_data.tail()

# 3. Beregn Manuel Lighed
Du har arbejdet hårdt på at forarbejde dataene! Nu er det enkelt at beregne ligheden mellem et
par chokolader, fordi alle træk er numeriske og i
samme område. For to chokolader skal du blot finde den kvadratrods middelkvadratfejl
(RMSE) for alle træk.

Kør først denne kode for at definere lighedsfunktionen.


In [ ]:
def getSimilarity(obj1, obj2):
  len1 = len(obj1.index)
  len2 = len(obj2.index)
  if not (len1 == len2):
    print "Error: Compared objects must have same number of features."
    sys.exit()
    return 0
  else:
    similarity = obj1 - obj2
    similarity = np.sum((similarity**2.0) / 10.0)
    similarity = 1 - math.sqrt(similarity)
    return similarity

Beregn nu ligheden mellem den første chokolade og de næste 4
chokolader. Bekræft den beregnede lighed i forhold til dine intuitive forventninger
ved at sammenligne den beregnede lighed med de faktiske trækdata, der vises i
næste celle.

Hvis du er nysgerrig efter ligheder mellem andre chokolader, kan du ændre
koden nedenfor og tage et kig!


In [ ]:
choc1 = 0  #@param
chocsToCompare = [1, 4]  #@param

print "Similarity between chocolates " + str(choc1) + " and ..."

for ii in range(chocsToCompare[0], chocsToCompare[1] + 1):
  print str(ii) + ": " + str(
      getSimilarity(choc_data.loc[choc1], choc_data.loc[ii]))

print "\n\nFeature data for chocolate " + str(choc1)
print choc_data_backup.loc[choc1:choc1, :]
print "\n\nFeature data for compared chocolates " + str(chocsToCompare)
print choc_data_backup.loc[chocsToCompare[0]:chocsToCompare[1], :]

# 4. Klynge Chokoladedataset

Vi er klar til at klynge chokoladerne! Kør koden for at opsætte k-means
klyngefunktionerne. Du behøver ikke at forstå koden.

**Bemærk**: Hvis du følger selvstudiet, skal du inden du kører resten af
dette Colab-læringsmiljø læse afsnittene om
[k-means](https://developers.google.com/machine-learning/clustering/algorithm/run-algorithm)
og
[kvalitetsmålinger](https://developers.google.com/machine-learning/clustering/interpret).


In [ ]:
#@title Run cell to setup functions
def dfSimilarity(df, centroids):
  ### dfSimilarity = Calculate similarities for dataframe input
  ### We need to calculate ||a-b||^2 = |a|^2 + |b|^2 - 2*|a|*|b|
  ### Implement this with matrix operations
  ### See the Appendix for further explanation
  numPoints = len(df.index)
  numCentroids = len(centroids.index)
  ## Strictly speaking, we don't need to calculate the norm of points
  # because it adds a constant bias to distances
  # But calculating it so that the similarity doesn't go negative
  # And that we expect similarities in [0,1] which aids debugging
  pointNorms = np.square(nla.norm(df, axis=1))
  pointNorms = np.reshape(pointNorms, [numPoints, 1])
  ## Calculate the norm of centroids
  centroidNorms = np.square(nla.norm(centroids, axis=1))
  centroidNorms = np.reshape(centroidNorms, (1, numCentroids))
  ## Calculate |a|^2 + |b|^2 - 2*|a|*|b|
  similarities = pointNorms + centroidNorms - 2.0 * np.dot(
      df, np.transpose(centroids))
  # Divide by the number of features
  # Which is 10 because the one-hot encoding means the "Maker" and "Bean" are
  # weighted twice
  similarities = similarities / 10.0
  # numerical artifacts lead to negligible but negative values that go to NaN on the root
  similarities = similarities.clip(min=0.0)
  # Square root since it's ||a-b||^2
  similarities = np.sqrt(similarities)
  return similarities


def initCentroids(df, k, feature_cols):
  # Pick 'k' examples are random to serve as initial centroids
  limit = len(df.index)
  centroids_key = np.random.randint(0, limit - 1, k)
  centroids = df.loc[centroids_key, feature_cols].copy(deep=True)
  # the indexes get copied over so reset them
  centroids.reset_index(drop=True, inplace=True)
  return centroids


def pt2centroid(df, centroids, feature_cols):
  ### Calculate similarities between all points and centroids
  ### And assign points to the closest centroid + save that distance
  numCentroids = len(centroids.index)
  numExamples = len(df.index)
  # dfSimilarity = Calculate similarities for dataframe input
  dist = dfSimilarity(df.loc[:, feature_cols], centroids.loc[:, feature_cols])
  df.loc[:, 'centroid'] = np.argmin(dist, axis=1)  # closest centroid
  df.loc[:, 'pt2centroid'] = np.min(dist, axis=1)  # minimum distance
  return df


def recomputeCentroids(df, centroids, feature_cols):
  ### For every centroid, recompute it as an average of the points
  ### assigned to it
  numCentroids = len(centroids.index)
  for cen in range(numCentroids):
    dfSubset = df.loc[df['centroid'] == cen,
                      feature_cols]  # all points for centroid
    if not (dfSubset.empty):  # if there are points assigned to the centroid
      clusterAvg = np.sum(dfSubset) / len(dfSubset.index)
      centroids.loc[cen] = clusterAvg
  return centroids


def kmeans(df, k, feature_cols, verbose):
  flagConvergence = False
  maxIter = 100
  iter = 0  # ensure kmeans doesn't run for ever
  centroids = initCentroids(df, k, feature_cols)
  while not (flagConvergence):
    iter += 1
    #Save old mapping of points to centroids
    oldMapping = df['centroid'].copy(deep=True)
    # Perform k-means
    df = pt2centroid(df, centroids, feature_cols)
    centroids = recomputeCentroids(df, centroids, feature_cols)
    # Check convergence by comparing [oldMapping, newMapping]
    newMapping = df['centroid']
    flagConvergence = all(oldMapping == newMapping)
    if verbose == 1:
      print 'Total distance:' + str(np.sum(df['pt2centroid']))
    if (iter > maxIter):
      print 'k-means did not converge! Reached maximum iteration limit of ' \
            + str(maxIter) + '.'
      sys.exit()
      return
  print 'k-means converged for ' + str(k) + ' clusters' + \
        ' after ' + str(iter) + ' iterations!'
  return [df, centroids]

Kør cellen for at klynge chokoladedatasættet, hvor `k` er antallet af
klynger.

På hver iteration af k-means viser output, hvordan summen af afstande fra alle eksempler til deres centroider reduceres, således at k-means altid konvergerer. Den følgende tabel viser data for de første chokolader. Længst til højre i tabellen skal du kontrollere den tildelte centroid for hvert eksempel i kolonnen `centroid` og afstanden fra eksemplet til dens centroid i kolonnen `pt2centroid`.


In [ ]:
k = 30  #@param

feature_cols = choc_data.columns.values  # save original columns
# initialize every point to an impossible value, the k+1 cluster
choc_data['centroid'] = k
# init the point to centroid distance to an impossible value "2" (>1)
choc_data['pt2centroid'] = 2
[choc_data, centroids] = kmeans(choc_data, k, feature_cols, 1)
print("Data for the first few chocolates, with 'centroid' and 'pt2centroid' on"
      ' the extreme right:')
choc_data.head()

## Gennemse Klynge Resultat
Gennemse chokoladerne i forskellige klynger ved at ændre parametret `clusterNumber`
i den næste celle og køre cellen. Overvej disse spørgsmål, mens du undersøger klyngerne:

*   Er klyngerne meningsfulde?
*   Vejer klyngerne visse træk mere end andre? Hvorfor?
*   Gør ændring af antallet af klynger klyngerne mere eller mindre
    meningsfulde?

Efter at have overvejet disse spørgsmål, kan du udvide næste afsnit for en diskussion af klynge-resultaterne.


In [ ]:
clusterNumber = 7  #@param
choc_data_backup.loc[choc_data['centroid'] == clusterNumber, :]

### Løsning: Diskussion af Clustering Resultater

Klik nedenfor for svaret.
**Diskussion**: Klynge-resultatet vægter visse træk mere end andre uden hensigt.

Dette skyldes, at en given chokoladeproducent vil have samme produktionsland, hvilket fører til gensidig information mellem trækkene `maker`,
`maker_lat` og `maker_long`. På samme måde, hvis hvert land har tendens til at dyrke en
bestemt type bønne, er der gensidig information mellem `origin_lat`,
`origin_long` og `bean_type`.

Som følge heraf vægter træk, der deler gensidig information, effektivt mere end uafhængige træk. Løsningen er at bruge en overvåget lighedsmåling, fordi DNN eliminerer korreleret information. Se
[k-means fordele og ulemper](https://developers.google.com/machine-learning/clustering/algorithm/advantages-disadvantages).

Nu skal du overveje one-hot encoding. Chokolader, der er lavet af forskellige producenter, vil
afvige med 1 i to kolonner. På samme måde vil chokolader, der er lavet af forskellige
bønnetyper, afvige med 1 i to træk. Derfor vil forskelle i producenter og bønnetyper blive vægtet dobbelt så meget som andre træk. Denne skæve vægtning fordrejer klynge-resultatet.


# 5. Kvalitetsmetrikker for Klynger

For klyngerne skal vi beregne de metrikker, der er diskuteret i
[Fortolk Resultater](https://developers.google.com/machine-learning/clustering/interpret).
Læs det kursusindhold, inden du begynder på dette kodeafsnit.

Kør den næste celle for at opsætte funktioner.


In [ ]:
#@title Run cell to set up functions { display-mode: "form" }
def clusterCardinality(df):
  k = np.max(df['centroid']) + 1
  k = k.astype(int)
  print 'Number of clusters:' + str(k)
  clCard = np.zeros(k)
  for kk in range(k):
    clCard[kk] = np.sum(df['centroid'] == kk)
  clCard = clCard.astype(int)
  # print "Cluster Cardinality:"+str(clCard)
  plt.figure()
  plt.bar(range(k), clCard)
  plt.title('Cluster Cardinality')
  plt.xlabel('Cluster Number: ' + str(0) + ' to ' + str(k - 1))
  plt.ylabel('Points in Cluster')
  return clCard


def clusterMagnitude(df):
  k = np.max(df['centroid']) + 1
  k = k.astype(int)
  cl = np.zeros(k)
  clMag = np.zeros(k)
  for kk in range(k):
    idx = np.where(df['centroid'] == kk)
    idx = idx[0]
    clMag[kk] = np.sum(df.loc[idx, 'pt2centroid'])
  # print "Cluster Magnitude:",clMag #precision set using np pref
  plt.figure()
  plt.bar(range(k), clMag)
  plt.title('Cluster Magnitude')
  plt.xlabel('Cluster Number: ' + str(0) + ' to ' + str(k - 1))
  plt.ylabel('Total Point-to-Centroid Distance')
  return clMag


def plotCardVsMag(clCard, clMag):
  plt.figure()
  plt.scatter(clCard, clMag)
  plt.xlim(xmin=0)
  plt.ylim(ymin=0)
  plt.title('Magnitude vs Cardinality')
  plt.ylabel('Magnitude')
  plt.xlabel('Cardinality')


def clusterQualityMetrics(df):
  clCard = clusterCardinality(df)
  clMag = clusterMagnitude(df)
  plotCardVsMag(clCard, clMag)

Beregn følgende metrikker ved at køre den næste celle:

*   kardinaliteten af dine klynger
*   størrelsen af dine klynger
*   kardinalitet kontra størrelse

Fra plot skal du finde klynger, der er outliers, og klynger, der er gennemsnitlige.
Sammenlign eksempler i outlier-klynger med dem i gennemsnitlige klynger ved at ændre `clusterNumber` i det foregående afsnit.


In [ ]:
clusterQualityMetrics(choc_data)

## Find det optimale antal klynger

Du vil finde det rigtige antal klynger, som du gjorde i den tidligere
programmeringsøvelse. For detaljer, læs "*Trin tre: Optimalt antal
klynger*" på siden
[Fortolk Resultater](https://developers.google.com/machine-learning/clustering/interpret).

Kør koden nedenfor. Følger plottet formen vist på "*Fortolk Resultater*"? Hvad er det
optimale antal klynger? Eksperimenter med parametrene nedenfor, hvis det er nødvendigt. Efter at have overvejet spørgsmålene, kan du udvide næste afsnit for en diskussion.


### Løsning: Diskussion om det optimale antal klynger

Klik nedenfor for løsningen.


In [ ]:
# Plot loss vs number of clusters
def lossVsClusters(kmin, kmax, kstep, choc_data):
  kmax += 1  # include kmax-th cluster in range
  kRange = range(kmin, kmax, kstep)
  loss = np.zeros(len(kRange))
  lossCtr = 0
  for kk in kRange:
    [choc_data, centroids] = kmeans(choc_data, kk, feature_cols, 0)
    loss[lossCtr] = np.sum(choc_data['pt2centroid'])
    lossCtr += 1
  plt.scatter(kRange, loss)
  plt.title('Loss vs Clusters Used')
  plt.xlabel('Number of clusters')
  plt.ylabel('Total Point-to-Centroid Distance')


kmin = 5  # @param
kmax = 80  # @param
kstep = 2  # @param
lossVsClusters(kmin, kmax, kstep, choc_data)

**Diskussion**: Den ideelle graf over tab versus antal klynger har et tydeligt vendepunkt, ud over hvilket tabets fald flader ud. Her mangler grafen et tydeligt vendepunkt. Dog flader tabet ud to gange, ved cirka `k = 15`
og `k = 35`, hvilket antyder, at `k` har optimale værdier tæt på 15 og 35. Bemærk, at din graf kan variere på grund af den iboende tilfældighed i k-means algoritmen.

Du
ser typisk en graf med et tydeligt vendepunkt, når data naturligt danner klumper.
Når data ikke har naturlige klumper, giver denne graf kun en antydning
om den optimale værdi for `k`.


## Diskussion

På siden
[Superviseret Lighedsmåling](https://developers.google.com/machine-learning/clustering/similarity/supervised-similarity),
læs "*Comparison of Manual and Supervised Measures*". Forsøg at forbinde beskrivelsen af en manuel lighedsmåling med det, du har lært i denne kodeøvelse. Klik derefter nedenfor for at se diskussionen. Til sidst, **hold denne Colab åben** for at sammenligne resultaterne med den næste Colab, der bruger en superviseret lighedsmåling.


Colab'en demonstrerer følgende karakteristika ved en manuel lighedsmåling:

*   **Eliminerer ikke overflødig information i korrelerede træk**. Som
    diskuteret i dette [afsnit](#scrollTo=MJtuP9w5jJHq), eliminerede vores manuelle lighedsmåling ikke overflødig information mellem træk.
*   **Giver indsigt i beregnede ligheder**. Ved at se på klynge-resultaterne kunne du se, hvordan producentens placering og bønnens oprindelse havde en større indflydelse på klynge-resultatet. Du så, hvordan one-hot encoding
    resulterede i, at producent og bønnetype blev vægtet dobbelt så meget som andre træk.
*   **Egnet til små datasæt med få træk**. Ja, du kunne nemt
    konstruere en manuel lighedsmåling for chokoladedatasættet, da det har
    færre end to tusind eksempler og kun ni træk.
*   **Ikke egnet til store datasæt med mange træk**. Hvis chokoladedatasættet
    havde dusinvis af træk og mange tusind eksempler, ville det være vanskeligt
    at konstruere en korrekt lighedsmåling og derefter verificere lighedsmålingen
    på tværs af datasættet.
